In [ ]:
# Author: Jin Zhang

In [ ]:
from __future__ import print_function, division

In [ ]:
import json
import os
import pandas as pd
#from collections import Counter
import numpy as np
import re 
% matplotlib inline


In [ ]:
pd.set_option('display.max_columns', 500)
pd.options.display.float_format = '{:,.3f}'.format

In [ ]:
master_json_file = './data/consolidated/master_listing.json'
with open(master_json_file, 'r') as fp:
    master_dict = json.load(fp)

In [ ]:
master_df = pd.DataFrame.from_dict(master_dict, orient='index')

In [ ]:
master_df.info()

In [ ]:
car_json_file = './data/consolidated/car_listing.json'
with open(car_json_file, 'r') as fp:
    car_dict = json.load(fp)

In [ ]:
car_df = pd.DataFrame.from_dict(car_dict, orient='index')
car_df = car_df.drop('bodyStyle', 1)


In [ ]:
car_df.head()

In [ ]:
# merge data

In [ ]:
df = car_df.merge(master_df,left_index = True, right_index = True, how = 'left')
df = df[df.index != 'null']

In [ ]:
df.info()

In [ ]:
# delete car make that has too small observations - either too expensive or not manufacturing anymore
# cut off is 20 observations

In [ ]:
df['make_count'] = df.groupby(['make'])['listPrice'].transform('count')

In [ ]:
df = df[df.make_count>=20]

In [ ]:
# Clean variables 

In [ ]:
driveTraindict = {'Four Wheel Drive':'AWD/4WD',
                  'Rear-Wheel Drive':'RWD',
                  'Front Wheel Drive':'FWD',
                  'Front-wheel Drive':'FWD',
                  'F':'FWD',
                  'All Wheel Drive':'AWD/4WD',
                  'All-wheel Drive':'AWD/4WD',
                  'quattro':'AWD/4WD',
                  'Rear Wheel Drive':'RWD',
                  'AWD':'AWD/4WD',
                  '4WD':'AWD/4WD',
                  ' ':'FWD',
                  'FWD':'FWD',
                  'RWD':'RWD'
                      }
df['driveTrain_clean'] = df['driveTrain'].map(driveTraindict)
df.driveTrain_clean.unique()
                      


In [ ]:
def get_transmission_type(transmission_list):
    result = []
    for item in transmission_list:
        lower_item = item.lower()
        if any(i in lower_item for i in ['cvt','continuously','continously']):
            result.append('CVT')
        elif any(i in lower_item for i in ['auto-shift manual','7-speed smg']):
            result.append('Semi-Auto')
        elif any(i in lower_item for i in ['automatic','auto','a/t','variable','6-speed','rental','4-speed','7-speed','5-speed tiptronic'])\
             or lower_item == 'a':
            result.append('Automatic')
        elif any(i in lower_item for i in ['manual','m/t','manuel']) or lower_item == 'm':
            result.append('Manual')
        elif any(i in lower_item for i in ['dual clutch','dual-clutch','double-clutch','7-speed porsche doppelkupplung']):
            result.append('Dual Clutch')
        elif any(i in lower_item for i in ['single-speed','single speed','not determined','1-speed']):
            result.append('Single Speed')
        else:
            result.append('N/A')
    return result



In [ ]:
df['transmission_clean'] = get_transmission_type(df.transmission)
df['transmission_clean'].unique()

In [ ]:
def get_engine_cyl_type(engine_list):
    result = []
    for item in engine_list:
        lower_item = item.lower()
        if any(i in lower_item for i in ['3 cyl','i3','i-3','3 cylinders']):
            result.append(3)
        elif any(i in lower_item for i in ['4 cyl','i-4','4 cylinders','i4','4-cyl','4 cylinder','l4','h-4','flat 4']):
            result.append(4)
        elif any(i in lower_item for i in ['v16','16 cylinders','v-16','16v']):
            result.append(16)
        elif any(i in lower_item for i in ['5 cyl','5 cylinders','5-cyl','5 cylinder','i-5','i5']) or lower_item == '5':
            result.append(5)
        elif any(i in lower_item for i in ['6 cyl','6 cylinders','v-6','v6','6-cyl','i6','h-6','i-6','flat 6','inline 6']):
            result.append(6)
        elif any(i in lower_item for i in ['8 cyl','8 cylinders','v8','v-8']):
            result.append(8)
        elif any(i in lower_item for i in ['10 cyl','10 cylinders','v-10','v10']):
            result.append(10)
        elif any(i in lower_item for i in ['v12','12 cylinders','v-12']):
            result.append(12)
        elif any(i in lower_item for i in ['react-text','not determined']):
            result.append(0)
        else:
            result.append(np.nan)
    return result

In [ ]:
df['engine_cyl_clean'] = get_engine_cyl_type(df.engine)

In [ ]:
df.loc[pd.isnull(df.engine_cyl_clean) & df.make.isin(['Cadillac','Buick']),'engine_cyl_clean'] = 4.0
df.loc[pd.isnull(df.engine_cyl_clean) & df.make.isin(['Volkswagen']),'engine_cyl_clean'] = 5.0
df.loc[pd.isnull(df.engine_cyl_clean) & df.make.isin(['Kia']) & df.make.isin(['Soul']),'engine_cyl_clean'] = 4.0
df.loc[pd.isnull(df.engine_cyl_clean),'engine_cyl_clean'] = 0.0


In [ ]:
df['engine_cyl_clean'].unique()

In [ ]:
def get_engine_fuel_type(engine_list):
    result = []
    for item in engine_list:
        lower_item = item.lower()
        if any(i in lower_item for i in ['gas/electric','hybrid']):
            result.append('Hybrid')
        elif any(i in lower_item for i in ['natural']):
            result.append('Natural_gas')
        elif any(i in lower_item for i in ['diesel']):
            result.append('Diesel')
        elif any(i in lower_item for i in ['flex fuel']):
            result.append('Flex')
        elif any(i in lower_item for i in ['electric','react-text']):
            result.append('Electric')
        elif any(i in lower_item for i in ['not determined']):
            result.append(np.nan)
            
        else:
            result.append('Gas')
    return result

In [ ]:
df['engine_fuel_type_clean'] = df['fuelType']

df.loc[pd.isnull(df.fuelType),'engine_fuel_type_clean'] = get_engine_fuel_type(df[pd.isnull(df.fuelType)]['engine'])

In [ ]:
df.loc[pd.isnull(df.engine_fuel_type_clean) & df.make.isin(['Subaru','Porsche']),'engine_fuel_type_clean'] = 'Gas'
df.loc[pd.isnull(df.engine_fuel_type_clean) & (~df.make.isin(['Subaru','Porsche'])),'engine_fuel_type_clean'] = 'Electric'
df = df[df.engine_fuel_type_clean !='Propane']
df.loc[df.make.isin(['Chevrolet']) & df.model.isin(['Volt']),'engine_fuel_type_clean'] = 'Hybrid'
df.loc[df.make.isin(['Toyota']) & df.model.isin(['RAV4 EV']),'engine_fuel_type_clean'] = 'Electric'
df.loc[df.make.isin(['FIAT']) & df.model.isin(['500e BATTERY ELECTRIC']),'engine_fuel_type_clean'] = 'Electric'
df.loc[df.make.isin(['Ford']) & df.model.isin(['Fusion Energi']),'engine_fuel_type_clean'] = 'Hybrid'
df.loc[df.make.isin(['Tesla']),'engine_fuel_type_clean'] = 'Electric'
df.loc[df.make.isin(['BMW']) & df.model.isin(['i3']),'engine_fuel_type_clean'] = 'Electric'


In [ ]:
df.engine_fuel_type_clean.unique()

In [ ]:
df['engine_turbo_clean'] = ['turbo' in x.lower() for x in df.engine]
df['engine_super_clean'] = ['super' in x.lower() for x in df.engine]

df[['engine_turbo_clean','engine_super_clean']] = df[['engine_turbo_clean','engine_super_clean']].astype(int)

In [ ]:
df.loc[df.mpgCity=='N/A','mpgCity'] = np.nan
df.loc[df.mpgHwy=='N/A','mpgHwy'] = np.nan
df[['mpgCity_clean','mpgHwy_clean']] = df.groupby(['make','model','year'])['mpgCity','mpgHwy'].transform(lambda x: x.fillna(x.mean()))
df[['mpgCity_clean','mpgHwy_clean']] = df.groupby(['make','model'])['mpgCity','mpgHwy'].transform(lambda x: x.fillna(x.mean()))

In [ ]:
df.loc[pd.isnull(df.mpgCity_clean) & df.engine_fuel_type_clean.isin(['Electric']),\
       ['mpgCity_clean','mpgHwy_clean']] = [300.0,300.0]
df.loc[pd.isnull(df.mpgCity_clean) & df.make.isin(['Jeep']),\
       ['mpgCity_clean','mpgHwy_clean']] = [19.0,26.0]
df.loc[pd.isnull(df.mpgCity_clean) & df.make.isin(['Dodge']),\
       ['mpgCity_clean','mpgHwy_clean']] = [16.0,22.0]
df.loc[pd.isnull(df.mpgCity_clean) & df.make.isin(['Ford']) & df.model.isin(['Transit Wagon','Expedition EL']),\
       ['mpgCity_clean','mpgHwy_clean']] = [15.0,19.0]
df.loc[pd.isnull(df.mpgCity_clean) & df.make.isin(['Ford']) & df.model.isin(['F-150']),\
       ['mpgCity_clean','mpgHwy_clean']] = [19.0,26.0]
df.loc[pd.isnull(df.mpgCity_clean) & df.make.isin(['Ford']),\
       ['mpgCity_clean','mpgHwy_clean']] = [10.0,10.0]
df.loc[pd.isnull(df.mpgCity_clean) & df.make.isin(['Cadillac']),\
       ['mpgCity_clean','mpgHwy_clean']] = [13.0,19.0]
df.loc[pd.isnull(df.mpgCity_clean) & df.make.isin(['Chevrolet']) & df.model.isin(['Malibu','Malibu Limited']),\
       ['mpgCity_clean','mpgHwy_clean']] = [27.0,36.0]
df.loc[pd.isnull(df.mpgCity_clean) & df.make.isin(['Chevrolet']) & df.model.isin(['Camaro']),\
       ['mpgCity_clean','mpgHwy_clean']] = [22.0,31.0]
df.loc[pd.isnull(df.mpgCity_clean) & df.make.isin(['Chevrolet']) & df.model.isin(['Express Cargo Van']),\
       ['mpgCity_clean','mpgHwy_clean']] = [11.0,17.0]
df.loc[pd.isnull(df.mpgCity_clean) & df.make.isin(['Chevrolet']) & df.model.isin(['Suburban']),\
       ['mpgCity_clean','mpgHwy_clean']] = [16.0,23.0]
df.loc[pd.isnull(df.mpgCity_clean) & df.make.isin(['Chevrolet']),\
       ['mpgCity_clean','mpgHwy_clean']] = [12.0,12.0]
df.loc[pd.isnull(df.mpgCity_clean) & df.make.isin(['GMC']) & df.model.isin(['Savana Cargo Van']),\
       ['mpgCity_clean','mpgHwy_clean']] = [11.0,17.0]
df.loc[pd.isnull(df.mpgCity_clean) & df.make.isin(['GMC']),\
       ['mpgCity_clean','mpgHwy_clean']] = [13.0,13.0]
df.loc[pd.isnull(df.mpgCity_clean) & df.make.isin(['Honda']),\
       ['mpgCity_clean','mpgHwy_clean']] = [27.0,38.0]
df.loc[pd.isnull(df.mpgCity_clean) & df.make.isin(['Nissan']),\
       ['mpgCity_clean','mpgHwy_clean']] = [10.0,13.0]
df.loc[pd.isnull(df.mpgCity_clean) & df.make.isin(['Volkswagen']),\
       ['mpgCity_clean','mpgHwy_clean']] = [23.0,34.0]
df.loc[pd.isnull(df.mpgCity_clean) & df.make.isin(['Buick']),\
       ['mpgCity_clean','mpgHwy_clean']] = [21.0,29.0]
df.loc[pd.isnull(df.mpgCity_clean) & df.make.isin(['Lincoln']),\
       ['mpgCity_clean','mpgHwy_clean']] = [16.0,24.0]
df.loc[pd.isnull(df.mpgCity_clean) & df.make.isin(['Toyota']),\
       ['mpgCity_clean','mpgHwy_clean']] = [78.0,74.0]
df.loc[pd.isnull(df.mpgCity_clean) & df.make.isin(['Ram']) & df.model.isin(['ProMaster Cargo Van']),\
       ['mpgCity_clean','mpgHwy_clean']] = [16.0,18.0]
df.loc[pd.isnull(df.mpgCity_clean) & df.make.isin(['Ram']),\
       ['mpgCity_clean','mpgHwy_clean']] = [13.0,13.0]
df.loc[pd.isnull(df.mpgCity_clean) & df.make.isin(['Mitsubishi']) & df.model.isin(['Galant']),\
       ['mpgCity_clean','mpgHwy_clean']] = [21.0,30.0]
df.loc[pd.isnull(df.mpgCity_clean) & df.make.isin(['Mitsubishi']) & df.model.isin(['Lancer Evolution']),\
       ['mpgCity_clean','mpgHwy_clean']] = [21.0,30.0]
df.loc[pd.isnull(df.mpgCity_clean) & df.make.isin(['Audi']) & df.model.isin(['A3 Sportback e-tron']),\
       ['mpgCity_clean','mpgHwy_clean']] = [61.0,87.0]
df.loc[pd.isnull(df.mpgCity_clean) & df.make.isin(['Audi']) & df.model.isin(['A4']),\
       ['mpgCity_clean','mpgHwy_clean']] = [27.0,37.0]
df.loc[pd.isnull(df.mpgCity_clean) & df.make.isin(['Audi']) & df.model.isin(['RS6']),\
       ['mpgCity_clean','mpgHwy_clean']] = [15.0,22.0]
df.loc[pd.isnull(df.mpgCity_clean) & df.make.isin(['BMW']) & df.model.isin(['1 Series']),\
       ['mpgCity_clean','mpgHwy_clean']] = [20.0,28.0]
df.loc[pd.isnull(df.mpgCity_clean) & df.make.isin(['BMW']) & df.model.isin(['3 Series']),\
       ['mpgCity_clean','mpgHwy_clean']] = [31.0,43.0]
df.loc[pd.isnull(df.mpgCity_clean) & df.make.isin(['BMW']) & df.model.isin(['6 Series']),\
       ['mpgCity_clean','mpgHwy_clean']] = [21.0,30.0]
df.loc[pd.isnull(df.mpgCity_clean) & df.make.isin(['BMW']) & df.model.isin(['M5','M6']),\
       ['mpgCity_clean','mpgHwy_clean']] = [15.0,22.0]
df.loc[pd.isnull(df.mpgCity_clean) & df.make.isin(['BMW']) & df.model.isin(['X5 eDrive']),\
       ['mpgCity_clean','mpgHwy_clean']] = [31.0,50.0]
df.loc[pd.isnull(df.mpgCity_clean) & df.make.isin(['BMW']) & df.model.isin(['i8']),\
       ['mpgCity_clean','mpgHwy_clean']] = [76.0,76.0]
df.loc[pd.isnull(df.mpgCity_clean) & df.make.isin(['INFINITI']) & df.model.isin(['G37 Coupe']),\
       ['mpgCity_clean','mpgHwy_clean']] = [19.0,27.0]
df.loc[pd.isnull(df.mpgCity_clean) & df.make.isin(['Kia']) & df.model.isin(['Soul']),\
       ['mpgCity_clean','mpgHwy_clean']] = [26.0,31.0]
df.loc[pd.isnull(df.mpgCity_clean) & df.make.isin(['Lexus']) & df.model.isin(['ES 300']),\
       ['mpgCity_clean','mpgHwy_clean']] = [21.0,29.0]
df.loc[pd.isnull(df.mpgCity_clean) & df.make.isin(['MINI']) & df.model.isin(['Cooper Paceman']),\
       ['mpgCity_clean','mpgHwy_clean']] = [27.0,32.0]
df.loc[pd.isnull(df.mpgCity_clean) & df.make.isin(['Mazda']) & df.model.isin(['MAZDA5']),\
       ['mpgCity_clean','mpgHwy_clean']] = [21.0,28.0]
df.loc[pd.isnull(df.mpgCity_clean) & df.make.isin(['Mazda']) & df.model.isin(['Mazda3']),\
       ['mpgCity_clean','mpgHwy_clean']] = [28.0,37.0]
df.loc[pd.isnull(df.mpgCity_clean) & df.make.isin(['Mazda']) & df.model.isin(['RX-8']),\
       ['mpgCity_clean','mpgHwy_clean']] = [16.0,23.0]
df.loc[pd.isnull(df.mpgCity_clean) & df.make.isin(['Mercedes-Benz']) & df.model.isin(['C-Class']),\
       ['mpgCity_clean','mpgHwy_clean']] = [24.0,34.0]
df.loc[pd.isnull(df.mpgCity_clean) & df.make.isin(['Mercedes-Benz']) & df.model.isin(['CLS']),\
       ['mpgCity_clean','mpgHwy_clean']] = [18.0,26.0]
df.loc[pd.isnull(df.mpgCity_clean) & df.make.isin(['Mercedes-Benz']) & df.model.isin(['GLC']),\
       ['mpgCity_clean','mpgHwy_clean']] = [22.0,27.0]
df.loc[pd.isnull(df.mpgCity_clean) & df.make.isin(['Mercedes-Benz']) & df.model.isin(['GLC']),\
       ['mpgCity_clean','mpgHwy_clean']] = [22.0,27.0]
df.loc[pd.isnull(df.mpgCity_clean) & df.make.isin(['Mercedes-Benz']),\
       ['mpgCity_clean','mpgHwy_clean']] = [16.0,20.0]
df.loc[pd.isnull(df.mpgCity_clean) & df.make.isin(['Porsche']) & df.model.isin(['718 Boxster','Boxster']),\
       ['mpgCity_clean','mpgHwy_clean']] = [21.0,28.0]
df.loc[pd.isnull(df.mpgCity_clean) & df.make.isin(['Porsche']) & df.model.isin(['911']),\
       ['mpgCity_clean','mpgHwy_clean']] = [22.0,30.0]
df.loc[pd.isnull(df.mpgCity_clean) & df.make.isin(['Porsche']) & df.model.isin(['Cayenne']),\
       ['mpgCity_clean','mpgHwy_clean']] = [20.0,24.0]
df.loc[pd.isnull(df.mpgCity_clean) & df.make.isin(['Subaru']),\
       ['mpgCity_clean','mpgHwy_clean']] = [20.0,27.0]
df.loc[pd.isnull(df.mpgCity_clean) & df.make.isin(['Volvo']),\
       ['mpgCity_clean','mpgHwy_clean']] = [19.0,28.0]

In [ ]:
df[['mpgCity_clean','mpgHwy_clean']] = df[['mpgCity_clean','mpgHwy_clean']].astype('float')
df[['mpgCity_clean','mpgHwy_clean']].describe()

In [ ]:
df.describe()

In [ ]:
def get_body_style(body_list):
    result = []
    for item in body_list:
        
        if item is None:
            result.append(np.nan)
        else:
            lower_item = item.lower()
            if any(i in lower_item for i in ['4d passenger van','van']):
                result.append('minivan/van')
            elif any(i in lower_item for i in ['4dr car']):
                result.append('sedan')
            elif any(i in lower_item for i in ['car']):
                result.append('wagon')
            elif any(i in lower_item for i in ['4 door suv']):
                result.append('suv')
            elif any(i in lower_item for i in ['hatchback']):
                result.append('hatchback')
            elif any(i in lower_item for i in ['coupe']):
                result.append('coupe')
            else:
                result.append(lower_item)
    
    return result
        


In [ ]:
df['bodyStyle_clean'] = get_body_style(df['bodyStyle'])

In [ ]:
df.loc[pd.isnull(df.bodyStyle_clean) & df.make.isin(['Chevrolet']),'bodyStyle_clean'] = 'sedan'
df.loc[pd.isnull(df.bodyStyle_clean) & df.make.isin(['Kia']),'bodyStyle_clean'] = 'wagon'

In [ ]:
df.bodyStyle_clean.unique()

In [ ]:
def get_exterior_Color(color_list):
    result = []
    for item in color_list:
        lower_item = item.lower()
        if item == 'N/A' or item == '-':
            result.append('not_available')
        
        elif any(i in lower_item for i in ['white','ivory','mojave','winter frost','cream','desert sand',\
                                           'creme','whitr', 'blizzard','starfire','arctic frost','platinum ice',\
                                          'moonglow','blizard','iceberg','capparis','lghtwht','moonlight pearl',\
                                           'bianco eldorado','fresh powder','pepper','alpine','bianco elderado']):
            result.append('white')
        elif any(i in lower_item for i in ['black','ebony','blk','blac','obsidian','jet','raven']):
            result.append('black')
        elif any(i in lower_item for i in ['blue','sapphire','still night','sky','indigo','sea','mntn air',\
                                          'mountain air','abyss','ocean','winter chill','ice storm','night',\
                                           'navy','cerulean','blazing','royal']):
            result.append('black')
        elif any(i in lower_item for i in ['brown','coffee','khaki','chocolate','tan','beige','forged bronze',\
                                           'fire agate','bronze','dark amber','truffle','biege','quicksand',\
                                          'mocha almond','espresso']):
            result.append('brown')
        elif any(i in lower_item for i in ['gold','champagne','palladium','champaigne','saharan stone',\
                                           'champaign','cashmere','sandstone','pyrite', 'saharan sun',\
                                           'beachwood','dune']):
            result.append('gold')
        elif any(i in lower_item for i in ['gray','grey','steel','graphite','metal','charcoal','anthracite',\
                                           'titanium','tungsten metallic','granit','guard','slate','hematite',\
                                          'gun','gary','smoky topaz','flint','galactic aqua','magnetic']):
            result.append('gray')
        elif any(i in lower_item for i in ['green','teal','tank','cactus','sublime','bamboo','spruce',\
                                           'sagebrush','jade','cypress','moss']):
            result.append('green')
        elif any(i in lower_item for i in ['yellow','spice']):
            result.append('yellow')
        elif any(i in lower_item for i in ['silver','billet','silvr','bronzemist','pewter','driftwood',\
                                           'lunar','moonstone','liquid platinum','iron frost','ipanema sand',\
                                          'silv','slvr','sivler','sliver','andesite','tungsten pearl','moonwalk']):
            result.append('silver')
        elif any(i in lower_item for i in ['maroon','cherry','wine','burgundy','deep auburn','burgrandy',\
                                           'burgandy','claret','tuscan sun','merlot','sangria','pomegranate',\
                                          'chardonnay','bordeaux']):
            result.append('maroon')
        elif any(i in lower_item for i in ['purple','plum']):
            result.append('purple')
        elif any(i in lower_item for i in ['red','crimson','ruby','rosso','chili','volcano']):
            result.append('red')
        elif any(i in lower_item for i in ['orange','sunset','tiger eye','forged copper','lava',\
                                           'volcanic','inferno','canyon copper','habanero','vitamin c',\
                                          'molten pearl','burnished copper']):
            result.append('orange')
        else:
            #print(lower_item)
            result.append('others')
    
    return result
        

In [ ]:
df['exteriorColor_clean'] = get_exterior_Color(df.exteriorColor)

In [ ]:
df.exteriorColor_clean.unique()

In [ ]:
def get_interior_Color(color_list):
    result = []
    for item in color_list:
        lower_item = item.lower()
        if item == 'N/A' or item == '-':
            result.append('not_available')
        
        elif any(i in lower_item for i in ['white','ivory','mojave','winter frost','cream','desert sand',\
                                           'creme','whitr', 'blizzard','starfire','arctic frost','platinum ice',\
                                          'moonglow','blizard','iceberg','capparis','lghtwht','moonlight pearl',\
                                           'bianco eldorado','fresh powder','pepper','alpine','bianco elderado']):
            result.append('white')
        elif any(i in lower_item for i in ['black','ebony','blk','blac','obsidian','jet','raven']):
            result.append('black')
        elif any(i in lower_item for i in ['blue','sapphire','still night','sky','indigo','sea','mntn air',\
                                          'mountain air','abyss','ocean','winter chill','ice storm','night',\
                                           'navy','cerulean','blazing','royal']):
            result.append('black')
        elif any(i in lower_item for i in ['brown','coffee','khaki','chocolate','tan','beige','forged bronze',\
                                           'fire agate','bronze','dark amber','truffle','biege','quicksand',\
                                          'almond','espresso','tan','taupe','stone','dune','hazelnut']):
            result.append('brown')
        
        elif any(i in lower_item for i in ['gray','grey','steel','graphite','metal','charcoal','anthracite',\
                                           'titanium','tungsten metallic','granit','guard','slate','hematite',\
                                          'gun','gary','smoky topaz','flint','galactic aqua','magnetic']):
            result.append('gray')
        elif any(i in lower_item for i in ['green','teal','tank','cactus','sublime','bamboo','spruce',\
                                           'sagebrush','jade','cypress','moss']):
            result.append('green')
        elif any(i in lower_item for i in ['yellow','spice']):
            result.append('yellow')
        
        elif any(i in lower_item for i in ['maroon','cherry','wine','burgundy','deep auburn','burgrandy',\
                                           'burgandy','claret','tuscan sun','merlot','sangria','pomegranate',\
                                          'chardonnay','bordeaux']):
            result.append('maroon')
        elif any(i in lower_item for i in ['purple','plum']):
            result.append('purple')
        elif any(i in lower_item for i in ['red','crimson','ruby','rosso','chili','volcano']):
            result.append('red')
        elif any(i in lower_item for i in ['orange','sunset','tiger eye','forged copper','lava',\
                                           'volcanic','inferno','canyon copper','habanero','vitamin c',\
                                          'molten pearl','burnished copper']):
            result.append('orange')
        else:
            #print(lower_item)
            result.append('others')
    
    return result
        

In [ ]:
df['interiorColor_clean'] = get_interior_Color(df.interiorColor)

In [ ]:
df.interiorColor_clean.unique()

In [ ]:
df['city_clean'] = df.city.str.lower()

In [ ]:
df['car_age_clean'] = 2018-df.year

In [ ]:
df.car_age_clean.describe()

In [ ]:
def get_keyword(keyword,feature_block):
    result = []
    feature_list = feature_block.lower().split(',')
    for item in feature_list:
        if keyword in item:
            result.append(item)
    return ','.join(result)


In [ ]:
df['feature_keyless_entry_clean'] = ['keyless entry' in item for item in df.features.str.lower()]
df['feature_keyless_start_clean'] = ['keyless start' in item for item in df.features.str.lower()]
df['feature_heated_seats_clean'] = [any(s in item for s in ['heated seats','heated front seats',\
                                                          'heated rear seats','heated &amp; ventilated front seats']) for item in df.features.str.lower()]
df['feature_ventilated_seats_clean'] = [any(s in item for s in ['ventilated seats','ventilated front seats']) for item in df.features.str.lower()]
df['feature_leather_seats_clean'] = [any(s in item for s in ['leather seat']) for item in df.features.str.lower()]
df['feature_rear_camera_parking_clean'] = [any(s in item for s in ['backup camera','back-up cameara',\
                                                           'back view camera','rearview camera',\
                                                           'rear-view camera','parking assist']) for item in df.features.str.lower()]
df['feature_driver_assist_clean'] = [any(s in item for s in ['driver assist','blind spot','blind-spot']) for item in df.features.str.lower()]
df['feature_cruise_control_clean'] = [any(s in item for s in ['cruise control']) for item in df.features.str.lower()]
df['feature_moonroof_clean'] = [any(s in item for s in ['moon roof','moonroof']) for item in df.features.str.lower()]
df['feature_navigation_clean'] = [any(s in item for s in ['navigation']) for item in df.features.str.lower()]
df['feature_premium_wheels_clean'] = [any(s in item for s in ['premium wheel']) for item in df.features.str.lower()]

In [ ]:
feature_cols = df.filter(regex=("^feature_.*clean$"),axis=1).columns
df[feature_cols] = df.filter(regex=("^feature_.*clean$"),axis=1).astype(float)

In [ ]:
df[feature_cols].describe()

In [ ]:
df['make_model_count'] = df.groupby(['make','model'])['listPrice'].transform('count')
df['model_clean'] = df['model']
df.loc[df['make_model_count'] < 20,'model_clean'] = 'others'
df.groupby(['make','model_clean'])['listPrice'].agg(['count'])
df['make_clean'] = df.make

In [ ]:
df.info()

In [ ]:
clean_data_cols = df.filter(regex=(".*clean$|listPrice|mileage"),axis=1).columns

In [ ]:
df[clean_data_cols].to_pickle('./analysis/truecar_clean_data.plk')